In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models

# ======================================================
# Device Setup: Use MPS if available (M1 Pro), CUDA if available, else CPU.
# ======================================================
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"[Device Setup] Using device: {device}\n")

# ======================================================
# 1. Set Up Paths and Load CSV
# ======================================================
base_dir = '/Users/bhaskarpramodchennupalli/Documents/Image_Detector'
csv_path = os.path.join(base_dir, 'train.csv')
train_data_dir = os.path.join(base_dir, 'train_data')

print("=== Step 1: Data Check and CSV Load ===")
df = pd.read_csv(csv_path)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
print(f"[Step 1] Loaded CSV file with {len(df)} records.")

# Optional: Check that every file exists in the train_data directory.
missing_files = []
for i, file_path in enumerate(df['file_name']):
    filename = os.path.basename(file_path)
    full_path = os.path.join(train_data_dir, filename)
    if not os.path.exists(full_path):
        missing_files.append(full_path)
    if i % 100 == 0:
        print(f"[Step 1] Checked {i} files...")
if missing_files:
    print(f"[Step 1] Missing {len(missing_files)} files:")
    for f in missing_files:
        print("    Missing:", f)
else:
    print("[Step 1] All files exist in the train_data directory!")
print("[Step 1] Data check complete.\n")

# ======================================================
# 2. Define the Custom Dataset Class (Top-Level)
# ======================================================
print("=== Step 2: Define Custom Dataset Class ===")
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        print(f"[Dataset Init] Loading CSV file: {csv_file}")
        self.data_frame = pd.read_csv(csv_file)
        if 'Unnamed: 0' in self.data_frame.columns:
            self.data_frame = self.data_frame.drop(columns=['Unnamed: 0'])
        self.transform = transform
        print(f"[Dataset Init] Loaded {len(self.data_frame)} records from {csv_file}")

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        file_path = self.data_frame.iloc[idx]['file_name']
        filename = os.path.basename(file_path)
        full_path = os.path.join(train_data_dir, filename)
        image = Image.open(full_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.data_frame.iloc[idx]['label']
        return image, label

print("[Step 2] CustomImageDataset class defined.\n")

# ======================================================
# 3. Define Candidate Model Architectures (Top-Level)
# ======================================================
print("=== Step 3: Define Candidate Model Architectures ===")

# Candidate 1: SimpleCNN (original model)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # After one pooling, 224x224 becomes 112x112.
        self.fc_layer = nn.Sequential(
            nn.Linear(16 * 112 * 112, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

# Candidate 2: DeeperCNN with extra layers
class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 32 x 224 x 224
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> 32 x 112 x 112
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # -> 64 x 112 x 112
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> 64 x 56 x 56
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Candidate 3: Pretrained ResNet18 fine-tuned for binary classification
def create_resnet18():
    weights = models.ResNet18_Weights.DEFAULT
    model = models.resnet18(weights=weights)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    return model

print("[Step 3] Candidate model architectures defined.")

# Create a dictionary for candidate models
candidates = {
    "SimpleCNN": SimpleCNN(),
    "DeeperCNN": DeeperCNN(),
    "ResNet18": create_resnet18()
}
print("[Step 3] Candidate models stored in dictionary.\n")

# ======================================================
# 4. Define Transformations & Create Dataset, Split, and DataLoaders (Top-Level)
# ======================================================
print("=== Step 4: Create Dataset, Split, and DataLoaders ===")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Uncomment if normalization is required:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print("[Step 4] Image transformations defined.")

print("[Step 4] Creating full dataset...")
full_dataset = CustomImageDataset(csv_file=csv_path, transform=transform)
print(f"[Step 4] Full dataset size: {len(full_dataset)}")

# Split dataset into training and validation (80/20 split)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
print(f"[Step 4] Training set size: {len(train_dataset)}")
print(f"[Step 4] Validation set size: {len(val_dataset)}")

batch_size = 32
print(f"[Step 4] Creating DataLoaders with batch size {batch_size} and num_workers=0 (for Jupyter)...")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("[Step 4] DataLoaders created.\n")

# ======================================================
# 5. Define Training and Evaluation Functions (Top-Level)
# ======================================================
print("=== Step 5: Define Training and Evaluation Functions ===")
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        avg_loss = running_loss / len(train_loader.dataset)
        
        # Evaluate on validation set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)
                outputs = model(images)
                preds = torch.round(torch.sigmoid(outputs))
                total += labels.size(0)
                correct += (preds == labels).sum().item()
        val_acc = correct / total
        print(f"[Step 5] Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
        best_val_acc = max(best_val_acc, val_acc)
    return best_val_acc

criterion = nn.BCEWithLogitsLoss()
print("[Step 5] Training and evaluation functions defined.\n")

# ======================================================
# 6. Train Candidate Models and Select the Best One
# ======================================================
print("=== Step 6: Training Candidate Models ===")
num_epochs = 5
results = {}
for name, model_candidate in candidates.items():
    print(f"\n[Step 6] Training candidate model: {name}")
    model_candidate.to(device)
    optimizer = optim.Adam(model_candidate.parameters(), lr=0.001)
    val_acc = train_and_evaluate(model_candidate, train_loader, val_loader, criterion, optimizer, num_epochs, device)
    results[name] = val_acc
    print(f"[Step 6] Candidate {name} achieved validation accuracy: {val_acc*100:.2f}%")
    
print("\n=== Step 6: Model Selection Results ===")
for name, acc in results.items():
    print(f"{name}: {acc*100:.2f}% validation accuracy")
best_model_name = max(results, key=results.get)
print(f"\n[Step 6] Best model: {best_model_name} with {results[best_model_name]*100:.2f}% validation accuracy")
print("=== Training and Model Selection Complete ===\n")


[Device Setup] Using device: mps

=== Step 1: Data Check and CSV Load ===
[Step 1] Loaded CSV file with 79950 records.
[Step 1] Checked 0 files...
[Step 1] Checked 100 files...
[Step 1] Checked 200 files...
[Step 1] Checked 300 files...
[Step 1] Checked 400 files...
[Step 1] Checked 500 files...
[Step 1] Checked 600 files...
[Step 1] Checked 700 files...
[Step 1] Checked 800 files...
[Step 1] Checked 900 files...
[Step 1] Checked 1000 files...
[Step 1] Checked 1100 files...
[Step 1] Checked 1200 files...
[Step 1] Checked 1300 files...
[Step 1] Checked 1400 files...
[Step 1] Checked 1500 files...
[Step 1] Checked 1600 files...
[Step 1] Checked 1700 files...
[Step 1] Checked 1800 files...
[Step 1] Checked 1900 files...
[Step 1] Checked 2000 files...
[Step 1] Checked 2100 files...
[Step 1] Checked 2200 files...
[Step 1] Checked 2300 files...
[Step 1] Checked 2400 files...
[Step 1] Checked 2500 files...
[Step 1] Checked 2600 files...
[Step 1] Checked 2700 files...
[Step 1] Checked 2800 fil

[Step 1] Checked 43500 files...
[Step 1] Checked 43600 files...
[Step 1] Checked 43700 files...
[Step 1] Checked 43800 files...
[Step 1] Checked 43900 files...
[Step 1] Checked 44000 files...
[Step 1] Checked 44100 files...
[Step 1] Checked 44200 files...
[Step 1] Checked 44300 files...
[Step 1] Checked 44400 files...
[Step 1] Checked 44500 files...
[Step 1] Checked 44600 files...
[Step 1] Checked 44700 files...
[Step 1] Checked 44800 files...
[Step 1] Checked 44900 files...
[Step 1] Checked 45000 files...
[Step 1] Checked 45100 files...
[Step 1] Checked 45200 files...
[Step 1] Checked 45300 files...
[Step 1] Checked 45400 files...
[Step 1] Checked 45500 files...
[Step 1] Checked 45600 files...
[Step 1] Checked 45700 files...
[Step 1] Checked 45800 files...
[Step 1] Checked 45900 files...
[Step 1] Checked 46000 files...
[Step 1] Checked 46100 files...
[Step 1] Checked 46200 files...
[Step 1] Checked 46300 files...
[Step 1] Checked 46400 files...
[Step 1] Checked 46500 files...
[Step 1]

[Step 3] Candidate models stored in dictionary.

=== Step 4: Create Dataset, Split, and DataLoaders ===
[Step 4] Image transformations defined.
[Step 4] Creating full dataset...
[Dataset Init] Loading CSV file: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/train.csv
[Dataset Init] Loaded 79950 records from /Users/bhaskarpramodchennupalli/Documents/Image_Detector/train.csv
[Step 4] Full dataset size: 79950
[Step 4] Training set size: 63960
[Step 4] Validation set size: 15990
[Step 4] Creating DataLoaders with batch size 32 and num_workers=0 (for Jupyter)...
[Step 4] DataLoaders created.

=== Step 5: Define Training and Evaluation Functions ===
[Step 5] Training and evaluation functions defined.

=== Step 6: Training Candidate Models ===

[Step 6] Training candidate model: SimpleCNN
[Step 5] Epoch 1/5 | Train Loss: 0.2961 | Val Acc: 90.79%
[Step 5] Epoch 2/5 | Train Loss: 0.1930 | Val Acc: 92.37%
[Step 5] Epoch 3/5 | Train Loss: 0.1357 | Val Acc: 93.52%
[Step 5] Epoch 4/5 | Tr

In [2]:
best_model = candidates["ResNet18"]
model_save_path = os.path.join(base_dir, 'best_model.pth')
torch.save(best_model.state_dict(), model_save_path)
print(f"Best model saved to {model_save_path}")

Best model saved to /Users/bhaskarpramodchennupalli/Documents/Image_Detector/best_model.pth
